# Data Consolidation Notebook



---



This notebook merges the following data:


*   Proteomic abundance values
*   Control protein abundance values
*   NetSurfP protein calculations
*   BioPython protein calculations
*   Nanoparticle properties
*   Wet lab experimental conditions

The result is an excel file that is used in the RFR and RFC code notebooks.


Import Statements
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import pickle
import openpyxl
from sklearn.preprocessing import LabelEncoder

Select Abundance Data
---

In [ ]:
# This is proteomic data from a core facility
data_dir = '/content/drive/MyDrive/ProteinCoronaPredict_PayneLab/Input_Data/Proteomics/Abundance_Files/'

# Abundance file options; these are created by Proteomic_Date_Perseus_to_df.ipynb
bov_swiss_files = ['Bov_Swiss_Intensity _original.xlsx'] # EDIT: modify list as needed based on file name and proteomics analysis type (i.e. Top3, iBAQ, Intensity)
#bov_swiss_trem_files = ['Bov Swiss + TrEMBLE Intensity .csv', 'Bov Swiss + TrEMBLE Top3 .csv', 'Bov Swiss + TrEMBLE iBAQ .csv']

# Specify which list of abundaces file you want
abund_files = bov_swiss_files

Specify file paths to the other data input files


*   Control_ file: proteomic reference sample (FBS no NP)
*   Biop_file: BioPython precalculations (find in input_data folder)
*   NSP_file: NSP precalculations (find in input_data folder)
*   NP_file: NP data corresponding to proteomic samples





In [4]:
controls_file = '/content/drive/MyDrive/ProteinCoronaPredict_PayneLab/Input_Data/Proteomics/Abundance_Files/controls_FBS_Intensity_v2.xlsx' # for Intensity
biop_file = '/content/drive/MyDrive/ProteinCoronaPredict_PayneLab/Input_Data/BioPython/Combined_biopyCalcs.xlsx'
nsp_file = '/content/drive/MyDrive/ProteinCoronaPredict_PayneLab/Input_Data/NetSurfP/Combined2.xlsx'
np_file = '/content/drive/MyDrive/ProteinCoronaPredict_PayneLab/Input_Data/Nanoparticles/NP_Database_BovOnly_v5.xlsx'

Read in Excel files in specified paths

In [5]:
controls = pd.read_excel(controls_file, header=0)

biop_data = pd.read_excel(biop_file, header=0)
biop_data.drop_duplicates(subset=['Entry'], inplace=True)

nsp_data = pd.read_excel(nsp_file)
nsp_data.drop_duplicates(subset=['Entry'], inplace=True)

np_data = pd.read_excel(np_file, header=0)

Data Merging
---

In [ ]:
for prot_file in abund_files:
  print(prot_file)
  name = prot_file[:-4]
  
  prot_file = data_dir + prot_file


  raw_MS_data = pd.read_excel(prot_file, header=0)


  raw_MS_data.drop(columns=['prot','prot2','MW'], inplace=True)
  raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'], var_name='Sample_num', value_name='Abundance')

  raw_MS_data.dropna(inplace=True)

  #Add abundance controls to the merged MS data
  MS_data_controls = pd.merge(raw_MS_data, controls, how='left', on='Entry')

  #Merge NetSurfP features and BioPython features
  raw_prop_data = pd.merge(MS_data_controls, biop_data, how='left', on='Entry')
  protein_data_complete = pd.merge(raw_prop_data, nsp_data, how='left', on='Entry')

  #Fill missing data with zeros
  protein_data_complete.fillna(0, inplace=True)

  #Add NP and wet lab experimental data
  protein_data_complete['Sample_num']=protein_data_complete['Sample_num'].astype(str)
  np_data["Sample_num"] = np_data["Sample_num"].astype(str)
  data_complete = pd.merge(protein_data_complete, np_data, how='inner', on='Sample_num')
  data_complete.fillna(0, inplace=True)


  #Columns to remove
  #From NetSurfP, remove 'exposed' entries and keep 'Exposed_exposed' only
  to_drop = data_complete.filter(like='total_exposed_')
  data_complete.drop(columns=to_drop, inplace=True)

  columns_to_drop = ["flexibility_var","flexibility_median","rsa_median","Ligands","Surface_Ligand","Shaken","ProteinID","Protein Source","Temperature",
                        "notes", "Notes", "Core Material", "Incubation Time (minutes)", "Ligands", "NPID", "Sequence", "Raw_FileID", "BatchID", ] 

  columns_to_drop = [col for col in columns_to_drop if col in data_complete.columns]
  data_complete.drop(columns=columns_to_drop, inplace=True)

  #Shuffle data to improve randomization of DataFrame
  data_complete = data_complete.sample(frac=1, random_state=42)

  #Save as Excel file
  # CHANGE TO YOUR NAME CONVENTION
  name_specific = 'v1'
  filename = f'/content/drive/MyDrive/ProteinCoronaPredict_PayneLab/Input_DataFrames/df_Bov Swiss Intensity_{name_specific}.xlsx'
  data_complete.to_excel(filename, index=False)
  print('done')


Bov_Swiss_Intensity _original.xlsx
done


The above file(s) is/are the ones that go directly into RFR and RFC Colab notebooks.